In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import classification_report

import random

RANDOM_SEED = 12345

In [ ]:
ros = RandomOverSampler(random_state=RANDOM_SEED)
rus = RandomUnderSampler(random_state=RANDOM_SEED)

def unbalanced_bagger(M, x, y):
    #M = Amount of base_learners, x = train set, y = class labels
    base_learners = []
    sampling_techniques = [ros, rus]
    for i in range(M):
        #boostrap replica
        rnd = np.random.choice(x.shape[0], size=x.shape[0])
        x_bootstrap = x.iloc[rnd,:] #WHY?
        y_bootstrap = y.iloc[rnd]
        #sample the replica with random sample technique
        technique = random.randint(0,len(sampling_techniques)-1)
        x_sample, y_sample = sampling_techniques[technique].fit_resample(x_bootstrap,y_bootstrap)
        #train
        clf = DecisionTreeClassifier()
        clf.fit(x_sample,y_sample)
        base_learners.append(clf)

    def predict(x):
        predictions = np.zeros([x.shape[0], len(base_learners)])
        print(predictions)
        for i in range(len(base_learners)):
            predictions[:,i] = base_learners[i].predict(x)
        return np.sign(np.sum(predictions, axis=1))
    return predict